In [0]:
%run "./Extractor_Functions"

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,broadcast,lead,row_number,collect_set

In [0]:
def AirpordsAfterApple():

    """Reading both Dataframes from Extract() module and saving into sepearte DFs"""
    Transaction_df,Customer_df = Extract()

    Transaction_df = Transaction_df.filter(Transaction_df['product_name'].isin('iPhone','AirPods')).\
            withColumn('Next_product',lead('product_name').\
                over(Window.partitionBy('customer_id').orderBy('transaction_date'))).\
                    filter(col('Next_product')=='AirPods').select('customer_id')
        
    joined_df = Customer_df.join(broadcast(Transaction_df),on='customer_id')
        
    print('Customers who bought Airpods after Iphone')
    joined_df.show()
    
    return (joined_df.select('customer_id','customer_name','location'))

def BothAirpodsIphone():

    """Reading both Dataframes from Extract() module and saving into sepearte DFs"""
    Transaction_df,Customer_df = Extract()

    Both_items = Transaction_df.filter(Transaction_df['product_name'].\
        isin('iPhone','AirPods')).select('customer_id').\
            dropDuplicates().rdd.flatMap(lambda x:x).collect()

    Transaction_df = Transaction_df.filter(Transaction_df['customer_id'].isin(Both_items))

    joined_df = Customer_df.join(broadcast(Transaction_df),on='customer_id').\
        select('customer_id','customer_name','location').dropDuplicates()
        
    print('Customers who bought both Airpods and Iphone')
    joined_df.show()

    return joined_df

def Productsafterfirstpurchase():

    """Reading both Dataframes from Extract() module and saving into sepearte DFs"""
    Transaction_df,Customer_df = Extract()

    Transaction_df = Transaction_df.withColumn('first',row_number().\
        over(Window.partitionBy('customer_id').orderBy('transaction_date'))).\
            filter(col('first') > 1).groupBy('customer_id').\
                agg(collect_set('product_name').alias('All_Products'))

    joined_df = Customer_df.join(broadcast(Transaction_df),on='customer_id').\
        select('customer_id','customer_name','location','All_Products').dropDuplicates()
        
    print('Customers who purchased below products after their first Purchase')
    joined_df.show()

    return joined_df